# Create json file of daily trips averaged by month

original table: `ptc.daily_trips`  

Processed to fill-in data gap using this query:  
```
SELECT to_char(dt, 'YYYY-MM') as period, 
     CASE WHEN to_char(dt, 'YYYY-MM') = '2018-03' THEN 108321
         ELSE avg(count)::integer 
     END as count 
FROM ptc.daily_trips
GROUP BY period
ORDER BY period
```

JSON format should be:  
```
{
	"tpd": {
		"keys": {
			"name": "years",
			"values": [
				"2016-09", "2016-10", "2016-11", "2016-12", "2017-01", "2017-02",
	       "2017-03", "2017-04", "2017-05", "2017-06", "2017-07", "2017-08",
	       "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02",
	       "2018-03", "2018-04", "2018-05", "2018-06", "2018-07", "2018-08",
	       "2018-09", "2018-10", "2018-11", "2018-12", "2019-01", "2019-02",
	       "2019-03"
			]
		},
		"city": [
			62242,  64443,  65595,  71107,  69280,  75834,  80486,  80165,
	        79951,  83822,  76303,  77527,  82788,  83752,  88247,  97224,
	        91856, 100055, 108321, 116587, 118861, 132495, 131914, 145230,
	       152776, 159412, 161733, 164054, 161267, 172254, 175803
		]
	}
}
```

**End of study date**: March 2019  
Anything after this date is to be considered for the **"post-study"** period  

In [1]:
import sys
sys.executable

'/home/nangini/anaconda3/bin/python'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker

from IPython.display import HTML, display
import datetime
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os

from datetime import datetime

import json

# Variables

In [3]:
study_end = '2019-03' # end of study period
study_bd = '2019-04' # need an overlap date for second line

# Read daily trip count from csv file
Created by the following query:  
```
SELECT to_char(dt, 'YYYY-MM') as period, 
     CASE WHEN to_char(dt, 'YYYY-MM') = '2018-03' THEN 108321
         ELSE avg(count)::integer 
     END as count 
FROM ptc.daily_trips
GROUP BY period
ORDER BY period
```

In [4]:
mydir = '../../../../Documents/PROJECTS/BDITTO/VFH/COT_PAGE/DATA/FROM_PGADMIN/'
# /home/nangini/Documents/PROJECTS/BDITTO/VFH/COT_PAGE/DATA/FROM_PGADMIN/daily_trips_processed.csv
# mydir = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/'

fname = mydir + 'daily_trips_processed.csv'


df = pd.read_csv(fname)

print(df.shape)
df.head(5)

(35, 2)


,period,count
0,2016-09,62242
1,2016-10,64443
2,2016-11,65595
3,2016-12,71107
4,2017-01,69280


In [5]:
df.tail(5)

,period,count
30,2019-03,175803
31,2019-04,169744
32,2019-05,172861
33,2019-06,177690
34,2019-07,160292


# Process data

In [6]:
# # Set dt col as index
# df.set_index('dt', inplace=True)

# # Convert index to datetime
# df.index = pd.to_datetime(df.index)

# # Group the data by month, and take the mean for each group (i.e. each month)
# # https://chrisalbon.com/python/data_wrangling/pandas_group_data_by_time/
# dm = df.resample('M').mean()

# # Store only YYYY-MM
# dm.index = dm.index.strftime('%Y-%m')

# # Convert count to int
# dm['count'] = dm['count'].astype('int')


In [7]:
# Set period col as index
df.set_index('period', inplace=True)

# Convert index to datetime
df.index = pd.to_datetime(df.index)

# Store only YYYY-MM
df.index = df.index.strftime('%Y-%m')

In [8]:
print(df.shape)
df.head(10)

(35, 1)


,count
2016-09,62242
2016-10,64443
2016-11,65595
2016-12,71107
2017-01,69280
2017-02,75834
2017-03,80486
2017-04,80165
2017-05,79951
2017-06,83822


In [9]:
# Round counts to nearest 100
import math
def roundup(x):
    return int(math.ceil(x / 100.0)) * 100

In [10]:
df['count'] = df['count'].apply(lambda x: roundup(x))

## Split into two tables
One for study period, one for post-study period

In [12]:
df.iloc[df.index > study_end]

,count
2019-04,169800
2019-05,172900
2019-06,177700
2019-07,160300


In [13]:
# Copy into two new dataframes (without altering original df)
df_s = pd.DataFrame()
df_ps = pd.DataFrame()

df_s = df.copy()
df_ps = df.copy()

In [14]:
# Null the values in df_ps that are BEFORE the end of the study period
df_ps.loc[df_ps.index < study_end, 'count'] = np.nan

df_ps.loc[df_ps.index < study_end]

,count
2016-09,NaN
2016-10,NaN
2016-11,NaN
2016-12,NaN
2017-01,NaN
2017-02,NaN
2017-03,NaN
2017-04,NaN
2017-05,NaN
2017-06,NaN


In [15]:
# Null the values in df_s that are AFTER the end of the study period
df_s.loc[df_s.index >= study_bd, 'count'] = np.nan

df_s

,count
2016-09,62300.0
2016-10,64500.0
2016-11,65600.0
2016-12,71200.0
2017-01,69300.0
2017-02,75900.0
2017-03,80500.0
2017-04,80200.0
2017-05,80000.0
2017-06,83900.0


# Create json object and write to file
Format:  
```
{
	"tpd": {
		"keys": {
			"name": "date",
			"values": [
				"2016-09", "2016-10", "2016-11", "2016-12", "2017-01", "2017-02",
	       "2017-03", "2017-04", "2017-05", "2017-06", "2017-07", "2017-08",
	       "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02",
	       "2018-03", "2018-04", "2018-05", "2018-06", "2018-07", "2018-08",
	       "2018-09", "2018-10", "2018-11", "2018-12", "2019-01", "2019-02",
	       "2019-03"
			]
		},
		"city": [
			62242,  64443,  65595,  71107,  69280,  75834,  80486,  80165,
	        79951,  83822,  76303,  77527,  82788,  83752,  88247,  97224,
	        91856, 100055, 108321, 116587, 118861, 132495, 131914, 145230,
	       152776, 159412, 161733, 164054, 161267, 172254, 175803
		]
	}
}

```

In [16]:
import json

def write_line_json(df1, df2, jname):
    o = {
        "tpd": {
            "keys": {
                "name": "date",
                "values": list(df1.index)
            },
            "city": [],
            'post-study': []
        }
    }
    
    row1=[]
    for val in df1['count']:
        if np.isnan(val):
            row1.append(None)
        else:
            row1.append(int(val))
            
    row2=[]
    for val in df2['count']:
        if np.isnan(val):
            row2.append(None)
        else:
            row2.append(int(val))

    o['tpd']['city'] = row1
    o['tpd']['post-study'] = row2

    file = open(jname, 'w')
    
    print(o)
    
    json.dump(o, file)

In [17]:
jname = '../src/data/fig1_dailytrips.json'
write_line_json(df_s, df_ps, jname)

{'tpd': {'keys': {'name': 'date', 'values': ['2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07']}, 'city': [62300, 64500, 65600, 71200, 69300, 75900, 80500, 80200, 80000, 83900, 76400, 77600, 82800, 83800, 88300, 97300, 91900, 100100, 108400, 116600, 118900, 132500, 132000, 145300, 152800, 159500, 172300, 164100, 161300, 172300, 175900, None, None, None, None], 'post-study': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 175900, 169800, 172900, 177700, 160300]}}
